# Leonardo's Ornithopter: Bio-Inspired Flight

> *"A bird is an instrument working according to mathematical law, which instrument it is within the capacity of man to reproduce with all its movements."*  
> — Leonardo da Vinci, Codex on the Flight of Birds

## Introduction
Leonardo da Vinci's ornithopter designs were based on his meticulous observations of birds. He understood that flight wasn't just about flapping wings up and down, but involved complex twisting and folding motions.

In this notebook, we explore the **unsteady aerodynamics** of flapping flight, using a bio-inspired model that includes:
1.  **Figure-8 Kinematics**: The wing tip traces a figure-8 path.
2.  **Theodorsen's Theory**: Accounts for the lag in lift generation (unsteady effects).
3.  **Membrane Elasticity**: The wing deforms under load, acting like a sail.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy import special

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Wing Kinematics
The wing motion is defined by three angles:
- $\phi(t)$: Stroke angle (flapping)
- $\theta(t)$: Deviation angle (up/down)
- $\alpha(t)$: Rotation angle (pitching)

$$ \phi(t) = \Phi \sin(\omega t) $$

### 2. Unsteady Aerodynamics (Theodorsen)
For flapping wings, the lift isn't instantaneous. We use Theodorsen's function $C(k)$ to model the circulatory lift:
$$ L = L_{qs} C(k) + L_{added\_mass} $$

Where $k = \frac{\omega c}{2V}$ is the reduced frequency.

### 3. Power Consumption
Flapping flight is energy-intensive. The power required is:
$$ P(t) = \text{Aerodynamic Power} + \text{Inertial Power} $$


In [ ]:
# Physics Constants
RHO_AIR = 1.225

def theodorsen_function(k):
    # Approximate magnitude of Theodorsen's function C(k)
    # Real part represents lift attenuation
    if k < 0.01: return 1.0
    H1 = special.hankel1(1, k)
    H0 = special.hankel1(0, k)
    C = H1 / (H1 + 1j * H0)
    return C.real

def simulate_flapping(freq_hz, span_m, chord_m, speed_ms):
    omega = 2 * np.pi * freq_hz
    t = np.linspace(0, 2.0 / freq_hz, 200) # 2 cycles
    
    # Kinematics
    stroke_amp = np.radians(45)
    phi = stroke_amp * np.sin(omega * t)
    phi_dot = stroke_amp * omega * np.cos(omega * t)
    
    # Aerodynamics
    area = span_m * chord_m
    k = omega * chord_m / (2 * speed_ms)
    C_k = theodorsen_function(k)
    
    # Quasi-steady lift (simplified)
    # Lift is proportional to velocity squared (forward + flapping)
    v_tip = phi_dot * span_m / 2
    v_total = np.sqrt(speed_ms**2 + v_tip**2)
    
    # Angle of attack variation (simplified)
    alpha_geom = np.radians(10)
    alpha_induced = np.arctan2(v_tip, speed_ms)
    alpha_eff = alpha_geom - alpha_induced
    
    cl = 2 * np.pi * alpha_eff
    lift_qs = 0.5 * RHO_AIR * area * v_total**2 * cl
    
    # Unsteady Lift
    lift_unsteady = lift_qs * C_k
    
    # Power (Drag * Velocity)
    cd = 0.05 + 0.1 * cl**2
    drag = 0.5 * RHO_AIR * area * v_total**2 * cd
    power = drag * v_total
    
    return t, lift_unsteady, power, phi, k

def plot_ornithopter(freq, span, speed):
    chord = span / 6.0 # Aspect ratio 6
    t, lift, power, phi, k = simulate_flapping(freq, span, chord, speed)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Lift & Power
    ax1.plot(t, lift, 'b-', label='Lift (N)')
    ax1.plot(t, power, 'r--', label='Power (W)')
    ax1.set_title(f'Performance (k={k:.2f})')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Force / Power')
    ax1.legend()
    ax1.axhline(0, color='black', linewidth=0.5)
    
    # Plot 2: Wing Tip Path (Visual)
    # Create a figure-8 path for visualization
    y = span/2 * np.cos(phi)
    z = span/2 * np.sin(phi)
    # Add deviation for figure-8
    deviation = 0.2 * np.sin(2 * np.pi * freq * t + np.pi/2)
    z += deviation
    
    ax2.plot(y, z, 'k-')
    ax2.set_title('Wing Tip Trajectory (Front View)')
    ax2.set_xlabel('Spanwise Position (m)')
    ax2.set_ylabel('Vertical Position (m)')
    ax2.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    avg_lift = np.mean(lift)
    avg_power = np.mean(power)
    print(f"Average Lift: {avg_lift:.1f} N")
    print(f"Average Power: {avg_power:.1f} W")
    print(f"Lift-to-Power Ratio: {avg_lift/avg_power:.2f} N/W")

interact(plot_ornithopter, 
         freq=widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='Freq (Hz)'),
         span=widgets.FloatSlider(min=2.0, max=12.0, step=0.5, value=8.0, description='Span (m)'),
         speed=widgets.FloatSlider(min=5.0, max=20.0, step=1.0, value=10.0, description='Speed (m/s)'));

## Conclusion

The simulation demonstrates that flapping flight requires significant power, especially at low speeds where unsteady effects dominate. Leonardo's designs, while mechanically ingenious, likely lacked the power-to-weight ratio required for human flight with the materials of his time.

However, his observation of the **figure-8 motion** was correct and is a fundamental principle of bird and insect flight.